Import modules:

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_dir = "/Users/simionmara/Desktop/neuroscience/allendata"
#Again, we might want to use another folder if we do want to overwrite potential data we have
#data_dir = "./converted"

#From here on everything works with both AllenSDK and the MiniSDK!
manifest_path = os.path.join(data_dir, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

List all available sessions with their properties:

In [4]:
sessions = cache.get_session_table() #Returns a Pandas dataframe
sessions.head() #Sneak peek of the table

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


We want to filter the sessions, in order to choose one that fits our analysis best. 

The things we consider when filtering: high nr of recorded units (we want one with many recorded units; let's say >500 because we want to have enough data), brain coverage in visual cortex (VISp, VISam, VISal, etc), balanced genotype (since some sessions target specific inhibitory or excitatory neurons, and this can cause bias neural responses), has sufficient recording time and trials. 

In [5]:
# Define filters
MIN_UNIT_COUNT = 500  # Minimum number of neurons recorded
PREFERRED_GENOTYPE = "wt/wt"  # Set to None if you want to include all genotypes

# Apply filtering
filtered_sessions = sessions[
    (sessions["unit_count"] >= MIN_UNIT_COUNT) &  # Select sessions with enough neurons
    (sessions["session_type"] == "brain_observatory_1.1")  # Ensure correct experiment type
]

# If genotype is a factor, filter further
if PREFERRED_GENOTYPE:
    filtered_sessions = filtered_sessions[filtered_sessions["full_genotype"] == PREFERRED_GENOTYPE]

# Sort sessions by unit count (descending order)
filtered_sessions = filtered_sessions.sort_values(by="unit_count", ascending=False)

# Display filtered sessions
print(f"Filtered {len(filtered_sessions)} sessions matching criteria")
print(filtered_sessions[["unit_count", "full_genotype", "ecephys_structure_acronyms"]])


Filtered 15 sessions matching criteria
           unit_count full_genotype  \
id                                    
757216464         959         wt/wt   
750332458         902         wt/wt   
742951821         893         wt/wt   
754829445         832         wt/wt   
732592105         824         wt/wt   
757970808         773         wt/wt   
750749662         761         wt/wt   
761418226         712         wt/wt   
744228101         659         wt/wt   
763673393         629         wt/wt   
739448407         625         wt/wt   
799864342         604         wt/wt   
737581020         568         wt/wt   
743475441         553         wt/wt   
754312389         502         wt/wt   

                                  ecephys_structure_acronyms  
id                                                            
757216464  [LP, DG, CA3, CA1, VISrl, nan, PO, LGd, HPF, V...  
750332458  [grey, VISrl, nan, VISal, IntG, IGL, LGd, CA3,...  
742951821  [VISal, nan, grey, VISl, VISrl, VI

Automatically select the best session:

In [6]:
# Find session 757216464 in the session table
session_id = 757216464

# Retrieve session properties
session_metadata = sessions.loc[session_id]

# Display session properties
print(session_metadata)


published_at                                               2019-10-03T00:00:00Z
specimen_id                                                           733457989
session_type                                              brain_observatory_1.1
age_in_days                                                               105.0
sex                                                                           M
full_genotype                                                             wt/wt
unit_count                                                                  959
channel_count                                                              2225
probe_count                                                                   6
ecephys_structure_acronyms    [LP, DG, CA3, CA1, VISrl, nan, PO, LGd, HPF, V...
Name: 757216464, dtype: object


In [7]:
# Extract key details
print(f"Session ID: {session_id}")
print(f"Number of Recorded Neurons: {session_metadata['unit_count']}")
print(f"Genotype: {session_metadata['full_genotype']}")
print(f"Brain Regions: {session_metadata['ecephys_structure_acronyms']}")
print(f"Session Type: {session_metadata['session_type']}")
print(f"Mouse Age (Days): {session_metadata['age_in_days']}")
print(f"Sex: {session_metadata['sex']}")


Session ID: 757216464
Number of Recorded Neurons: 959
Genotype: wt/wt
Brain Regions: ['LP' 'DG' 'CA3' 'CA1' 'VISrl' nan 'PO' 'LGd' 'HPF' 'VISal' 'TH' 'LGv'
 'VISl' 'MGm' 'SGN' 'ProS' 'SUB' 'VISp' 'Eth' 'APN' 'VIS' 'MRN' 'MB' 'NOT'
 'VISam']
Session Type: brain_observatory_1.1
Mouse Age (Days): 105.0
Sex: M


In [8]:
# Select the session ID
session_id = 757216464

# Download the session data
session_data = cache.get_session_data(session_id)

# Confirm it was downloaded
print(f"Session {session_id} successfully loaded!")


/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Session 757216464 successfully loaded!


We look at df which contains detailed metadata for every recorded neuron ('unit'). 

Each row represents one neuron and each column represents a property of that neuron. 

Among these properties, the relevant ones for our project are:

* firing_rate = how often that neuron fires, on average (spikes/ sec)

* d_prime, isolation_distance, L_ratio, isi_violations = quality metrics for spike sorting (how confident the algorithm is that the 'unit' corresponds to a real neuron); we can filter our low_quality neurons based on these properties

* ecephys_structure_acronym = brain region the neuron came from

In [9]:
session_data.units

/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because vers

,waveform_PT_ratio,waveform_amplitude,amplitude_cutoff,cluster_id,cumulative_drift,d_prime,firing_rate,isi_violations,isolation_distance,L_ratio,...,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,
951812724,0.530584,165.705150,0.064669,0,237.67,6.884674,9.194248,0.016578,99.937016,0.000019,...,128.0,MRN,8292.0,3663.0,6645.0,probeA,See electrode locations,29999.957750,1249.998240,True
951812745,0.442866,190.595730,0.000315,2,261.61,5.568195,8.493725,0.095506,65.697146,0.002596,...,128.0,MRN,8285.0,3638.0,6650.0,probeA,See electrode locations,29999.957750,1249.998240,True
951812785,0.361222,134.263155,0.000120,6,206.92,5.579259,24.437194,0.000587,86.525032,0.008332,...,128.0,MRN,8274.0,3604.0,6655.0,probeA,See electrode locations,29999.957750,1249.998240,True
951812765,0.760445,124.347795,0.002866,4,331.84,5.722753,2.015055,0.019174,65.450579,0.000766,...,128.0,MRN,8277.0,3612.0,6654.0,probeA,See electrode locations,29999.957750,1249.998240,True
951812811,0.563871,89.634870,0.002207,9,169.08,5.157968,37.454772,0.005522,507.037686,0.005300,...,128.0,MRN,8259.0,3552.0,6663.0,probeA,See electrode locations,29999.957750,1249.998240,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951804296,0.210923,306.674160,0.001616,197,41.65,5.697925,0.687647,0.000000,64.966837,0.000495,...,417.0,VISrl,7418.0,1140.0,8734.0,probeF,See electrode locations,30000.029774,1250.001241,True
951804325,0.319148,99.954465,0.000092,202,134.30,4.619626,2.477649,0.006341,80.954185,0.000225,...,417.0,VISrl,7413.0,1123.0,8742.0,probeF,See electrode locations,30000.029774,1250.001241,True
951804346,0.417369,215.272590,0.001321,206,69.69,8.488513,0.376695,0.000000,92.507609,0.000001,...,417.0,VISrl,7408.0,1105.0,8750.0,probeF,See electrode locations,30000.029774,1250.001241,True


In [23]:
# Number of neurons before filtering:
len(session_data.units)

959

We want to filter out the low quality neurons, because:

* some neurons have low firing rate (very little signal)

* high noise (unreliable spikes = the spike that we're seeing might not be the actual spike of the neuron)

* unstable waveform (may drift (which means that the recorded signal from a neuron changes position over time, which happens when the brain or the probe might move slightly during a long experiment) or not be isolated well (many neurons are firing from the same brain region, so their electrical activity overlaps))

So we apply the following threshold:

* for **firing_rate** >1Hz which excludes silent or barely active neurons (likely noise or poor data)

* for **isi_violations** <0.5 which ensures spikes aren’t overlapping unnaturally — cleaner spike sorting

* for **d_prime** >2 which measures how distinguishable the neuron is from noise — 2+ is strong

* for **amplitude_cutoff** <0.1 which helps remove unstable waveforms — good for reliability

* for **isolation_distance** >20 which indicates well-isolated neurons (less noise from other units nearby)

In [26]:
# Filtering the neurons based on certain thresholds:
good_units = session_data.units[
    (session_data.units["firing_rate"] > 1) & 
    (session_data.units["isi_violations"] < 0.5) &
    (session_data.units["d_prime"] > 2) &
    (session_data.units["amplitude_cutoff"] < 0.1) &
    (session_data.units["isolation_distance"] > 20)
]

# These are the unit_ids we'll use from now on
good_unit_ids = good_units.index.values

In [27]:
# Number of neurons after filtering
len(good_units)

865

Number of neurons recorded per brain region:

In [28]:
good_units["ecephys_structure_acronym"].value_counts()

LP       111
CA1      103
PO        78
VISp      70
VISam     59
VIS       47
APN       45
VISal     45
VISl      44
Eth       42
SUB       40
DG        39
SGN       38
VISrl     31
TH        17
MGm       13
LGv       11
CA3       11
ProS       6
NOT        5
MRN        5
MB         3
LGd        2
Name: ecephys_structure_acronym, dtype: int64

Do we have enough neurons in the visual cortex? We have 

VISp: 70 neurons

VISam: 59

VISal: 45

VISl: 44

VISrl: 31

Which makes 249 neurons from visual area, which is enough to fit encoding models, train ML decoders, compare performance vs neuron count.

Extract stimulus types and presentation times 

In [12]:
# Get unique stimuli presented in this session
stimuli = session_data.stimulus_presentations["stimulus_name"].unique()
print("Stimuli Presented in This Session:", stimuli)

/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Stimuli Presented in This Session: ['spontaneous' 'gabors' 'flashes' 'drifting_gratings'
 'natural_movie_three' 'natural_movie_one' 'static_gratings'
 'natural_scenes']


In [13]:
session_data.stimulus_presentations["stimulus_name"].value_counts()

natural_movie_three    36000
natural_movie_one      18000
static_gratings         6000
natural_scenes          5950
gabors                  3645
drifting_gratings        630
flashes                  150
spontaneous               15
Name: stimulus_name, dtype: int64

One trial = One presentation of a visual stimulus — like a single drifting grating with a certain contrast and temporal frequency.

Are 630 trials enough? Yes but with limitations. We can do region specific-decoding (like with 50-100 neurons per area) or we can pool them together and decode from the entire visual cortex. 

We can't use NNs for example, but simple ML is ok. 

Also, later we can scale up using multiple sessions for stronger generalization.

In [14]:
drifting = session_data.stimulus_presentations[
    session_data.stimulus_presentations["stimulus_name"] == "drifting_gratings"
]

drifting.head()


,stimulus_block,start_time,stop_time,frame,phase,temporal_frequency,x_position,color,spatial_frequency,size,y_position,stimulus_name,orientation,contrast,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
3798,2.0,1586.113585,1588.115245,null,"[42471.86666667, 42471.86666667]",1.0,null,null,0.04,"[250.0, 250.0]",null,drifting_gratings,90.0,0.8,2.00166,246
3799,2.0,1589.116095,1591.117775,null,"[42471.86666667, 42471.86666667]",8.0,null,null,0.04,"[250.0, 250.0]",null,drifting_gratings,0.0,0.8,2.00168,247
3800,2.0,1592.118605,1594.120275,null,"[42471.86666667, 42471.86666667]",8.0,null,null,0.04,"[250.0, 250.0]",null,drifting_gratings,90.0,0.8,2.00167,248
3801,2.0,1595.121125,1597.122785,null,"[42471.86666667, 42471.86666667]",15.0,null,null,0.04,"[250.0, 250.0]",null,drifting_gratings,135.0,0.8,2.00166,249
3802,2.0,1598.123625,1600.125295,null,"[42471.86666667, 42471.86666667]",1.0,null,null,0.04,"[250.0, 250.0]",null,drifting_gratings,315.0,0.8,2.00167,250


In [15]:
drifting = session_data.stimulus_presentations[
    session_data.stimulus_presentations["stimulus_name"] == "static_gratings"
]

drifting.head()

,stimulus_block,start_time,stop_time,frame,phase,temporal_frequency,x_position,color,spatial_frequency,size,y_position,stimulus_name,orientation,contrast,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
49434,8.0,5399.300205,5399.550405,null,0.5,null,null,null,0.32,"[250.0, 250.0]",null,static_gratings,0.0,0.8,0.250200,4787
49435,8.0,5399.550405,5399.800605,null,0.0,null,null,null,0.32,"[250.0, 250.0]",null,static_gratings,0.0,0.8,0.250200,4788
49436,8.0,5399.800605,5400.050805,null,0.5,null,null,null,0.04,"[250.0, 250.0]",null,static_gratings,90.0,0.8,0.250200,4789
49437,8.0,5400.050805,5400.301005,null,0.5,null,null,null,0.16,"[250.0, 250.0]",null,static_gratings,120.0,0.8,0.250200,4790
49438,8.0,5400.301005,5400.551212,null,0.25,null,null,null,0.08,"[250.0, 250.0]",null,static_gratings,90.0,0.8,0.250207,4791


We printed the first 5 drifting gratings trials.

Each row = one trial of drifting gratings trial

Each column = feature/ setting used in that trial.

For example: At 1586.11 seconds into the experiment, a drifting grating with 1 Hz temporal frequency, 0.8 contrast, and 90° orientation was shown, and lasted ~2 seconds.

These values (especially temporal_frequency or contrast) are our target for the decoding part. 

In [16]:
session_data.stimulus_presentations[
    session_data.stimulus_presentations["stimulus_name"] == "drifting_gratings"
][["temporal_frequency", "contrast"]].drop_duplicates()


,temporal_frequency,contrast
stimulus_presentation_id,,
3798,1.0,0.8
3799,8.0,0.8
3801,15.0,0.8
3806,null,null
3810,2.0,0.8
3813,4.0,0.8


Extract spike times 

In [17]:
print(f"Total Neurons Recorded: {len(session_data.units)}")
print("Brain Regions:", session_data.units["ecephys_structure_acronym"].unique())

Total Neurons Recorded: 959
Brain Regions: ['MRN' 'MB' 'APN' 'NOT' 'CA1' 'VISam' 'Eth' 'TH' 'DG' 'VIS' 'MGm' 'SGN'
 'LP' 'ProS' 'SUB' 'VISp' 'LGv' 'CA3' 'VISl' 'PO' 'LGd' 'VISal' 'VISrl']


Extracting stimulus presentations

In [18]:
# Extract all stimulus presentations
stimuli_df = session_data.stimulus_presentations

# Filter only "drifting_gratings"
drifting_gratings = stimuli_df[stimuli_df["stimulus_name"] == "drifting_gratings"]

# Display a few samples
print(drifting_gratings.head())


                         stimulus_block   start_time    stop_time frame  \
stimulus_presentation_id                                                  
3798                                2.0  1586.113585  1588.115245  null   
3799                                2.0  1589.116095  1591.117775  null   
3800                                2.0  1592.118605  1594.120275  null   
3801                                2.0  1595.121125  1597.122785  null   
3802                                2.0  1598.123625  1600.125295  null   

                                                     phase temporal_frequency  \
stimulus_presentation_id                                                        
3798                      [42471.86666667, 42471.86666667]                1.0   
3799                      [42471.86666667, 42471.86666667]                8.0   
3800                      [42471.86666667, 42471.86666667]                8.0   
3801                      [42471.86666667, 42471.86666667]           